## Data Augmentation

In [1]:
#load data
import convenience

df_train_val, sample_rates = convenience.load_train()
df_train_val['augmented_data'] = df_train_val.audio
df_train_val.head()

,file_name,stratify,accent,gender,audio,length,augmented_data
0,1f_1018.wav,1f,1,f,"[tensor(-0.0002), tensor(-6.1035e-05), tensor(...",3.413312,"[tensor(-0.0002), tensor(-6.1035e-05), tensor(..."
1,1f_1026.wav,1f,1,f,"[tensor(-0.0008), tensor(-0.0008), tensor(-0.0...",5.632000,"[tensor(-0.0008), tensor(-0.0008), tensor(-0.0..."
2,1f_1031.wav,1f,1,f,"[tensor(-0.0002), tensor(-0.0002), tensor(-0.0...",4.693313,"[tensor(-0.0002), tensor(-0.0002), tensor(-0.0..."
3,1f_1070.wav,1f,1,f,"[tensor(0.0022), tensor(0.0019), tensor(0.0018...",3.925312,"[tensor(0.0022), tensor(0.0019), tensor(0.0018..."
4,1f_1075.wav,1f,1,f,"[tensor(0.), tensor(3.0518e-05), tensor(0.), t...",2.737500,"[tensor(0.), tensor(3.0518e-05), tensor(0.), t..."


In [2]:
sr = list(sample_rates)[0]
sr

#initialize sr variable (sampling rate) 

16000

In [3]:
import torch

In [4]:
waveform = df_train_val.audio[0]
print(waveform)
waveform.shape

tensor([-2.1362e-04, -6.1035e-05, -3.0518e-05,  ...,  6.4087e-04,
         7.3242e-04,  5.7983e-04])


torch.Size([54613])

#### Librosa

In this part I apply different types of augmentation techniques. They're programmed in a pipeline, where the output of the previous is the input to the next one. 
If you want to apply the augmentation technique to a 'clean slate', apply it to df_train_val['augmented'].

Code is adapted from: https://www.kaggle.com/code/huseinzol05/sound-augmentation-librosa#apply-hpss 

(Please note it would be a good idea to see for which augmentations the model has the best performance, yet also a good generalizability!)

In [6]:
import numpy as np
import librosa 

import seaborn as sns 
sns.set() #iirc mostly for visuals 
import tensorflow as tf
from IPython.display import Audio

In [11]:
#pitch shifted audio
#This is the begining of the pipeline, so df_train_val is used. 
pitch_shift_audio = [] #I always create a seperate list to avoid overwriting the original variable
for file in df_train_val['augmented_data']:
    audio_pitch = file.numpy() #data has to be a numpy array for Librosa to work
    bins_per_octave = 12 
    pitch_pm = 2 
    pitch_change =  pitch_pm * 2*(np.random.uniform())  #every audiofile has a random pitch change (can be tweaked)
    
    pitch_shift_audio.append(librosa.effects.pitch_shift(audio_pitch, sr = 16000, n_steps=pitch_change, bins_per_octave=bins_per_octave)) 

print(pitch_shift_audio)

[array([-1.5918526e-04, -6.1902465e-05,  5.4227785e-06, ...,
        3.4041423e-04,  4.2316032e-04,  3.8433392e-04], dtype=float32), array([-0.00067767, -0.00083563, -0.00078593, ...,  0.00029237,
        0.00028909,  0.00028728], dtype=float32), array([-0.00022416, -0.00021992, -0.00014123, ..., -0.0004444 ,
       -0.0004203 , -0.00038201], dtype=float32), array([ 0.00196474,  0.00196314,  0.0017457 , ..., -0.00066577,
       -0.00061824, -0.0006555 ], dtype=float32), array([ 3.5153889e-06,  2.7358115e-05, -4.0950695e-06, ...,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00], dtype=float32), array([-5.0929282e-04, -6.0834287e-04, -5.1187596e-04, ...,
       -2.1417645e-05, -4.8923699e-05, -3.4982266e-05], dtype=float32), array([-2.0413145e-07,  8.6185321e-07, -1.8966995e-06, ...,
        0.0000000e+00,  0.0000000e+00,  0.0000000e+00], dtype=float32), array([-4.2057986e-04, -3.2277714e-04, -4.2671137e-04, ...,
        8.9998342e-05,  5.1367369e-05,  4.4694949e-05], dtype=float32

In [23]:
#len(pitch_shift_audio[0])
type(pitch_shift_audio[2])

numpy.ndarray

In [ ]:
#change speed
#notice how pitch_shift_audio is the input for this part (so no .numpy needed anymore)
speed_shift_audio = []
for file in pitch_shift_audio:
    audio_speed = file 
    speed_change = np.random.uniform(low=0.9,high=1.1) #strength of the effect (can be tweaked)
    tmp = librosa.effects.time_stretch(audio_speed, rate = speed_change) 
    minlen = min(audio_speed.shape[0], tmp.shape[0])
    audio_speed *= 0 
    audio_speed [0:minlen] = tmp[0:minlen] 
    
    speed_shift_audio.append(audio_speed)

In [ ]:
#distribution noise
noise_dist_audio = []

for file in speed_shift_audio:
    audio_noise = file
    noise_amp = 0.005*np.random.uniform()*np.amax(audio_noise) #random audio noise, can be changed to any distribution from https://docs.scipy.org/doc/numpy-1.13.0/reference/routines.random.html
    audio_noise = audio_noise + noise_amp * np.random.normal(size= audio_noise.shape[0]) #mathy math for noise
    
    noise_dist_audio.append(audio_noise) 

In [ ]:
#random shift
rand_shift_audio = []

for file in noise_dist_audio:
    audio_shift = file
    timeshift_fac = 0.2 *2*(np.random.uniform()-0.5)  # up to 20% of length shift (can be tweaked)
    start = int(audio_shift.shape[0] * timeshift_fac)
    if (start > 0): 
        audio_shift = np.pad(audio_shift,(start,0),mode='constant')[0:audio_shift.shape[0]]
    else:
        audio_shift = np.pad(audio_shift,(0,-start),mode='constant')[0:audio_shift.shape[0]]
    Audio(audio_shift, rate= sr)
    
    rand_shift_audio.append(audio_shift)

In [ ]:
#stretching
stretch_shift_audio = []

for file in rand_shift_audio:
    input_length = len(file)
    streching = file
    streching = librosa.effects.time_stretch(streching , rate = 1.1) #similar code to speed up due to file needing to fit audio
    if len(streching) > input_length:
        streching = streching[:input_length]
    else:
        streching = np.pad(streching, (0, max(0, input_length - len(streching))), "constant")

    stretch_shift_audio.append(streching)

In [19]:
#supposed to convert the augmented data back into the df, in tensor form, but I was unable to make it work, kept giving dimension errors
#there could be a hidden issue in the functions used where it changes the dimensions, I suspect speed or stretch 
#I would try to run all parts of the pipeline seperately and see from where the issue arrises 

##Bara here! I think the issue is because you're converting a list on np arrays, which i dont think the .tensor hnows how to handle?

#augmented_data = torch.tensor(pitch_shift_audio)

#df_train_val['augmented_data'] = augmented_data
#df_train_val.head()